In [2]:
from ml_config import *
from jw_package import *
pd.options.mode.chained_assignment = None

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [3]:
def missing_to_nan_weather(station_nbr):
    print('missing_to_nan started !')
    
    full_weather = load_weather()
    
    full_weather = full_weather[full_weather['station_nbr']==station_nbr]
    
    # sunrise, sunset, category는 연속형변수가 아니므로 일단 뺌
    weathers = set(list(full_weather.columns)[2:20])
    targets = weathers - {'sunset','sunrise','codesum'}
    
    def trim_str(x):
        import re
                            # 실수만 추출
        result = re.findall('(\d+(?:\.\d+)?)',x)
        if result:
            return float(result[0])
        return np.nan
    
    for weather in targets:
        full_weather.loc[:,weather] = full_weather.loc[:,weather].astype(str).apply(trim_str).astype(float)
    
    print('missing_to_nan completed !')
    return full_weather.reset_index(drop=True)

In [4]:
def how_many_missing(nan_weather):
    
    weathers = set(list(nan_weather.columns)[2:20])
    targets = weathers - {'sunset','sunrise','codesum'}
    
    how_many = dict()
    total = len(nan_weather)
    
    for weather in targets:
        not_missing = len(nan_weather[weather].dropna())
    
        msg = 'total = '+ str(total) +' '
        msg += ', missing = '+ str(total - not_missing)
        
        how_many[weather] = msg
    
    return how_many

In [75]:
def all_missing(nan_weather, weather):
    # 1. 날씨가 가장 비슷한 다른 station의 weather를 상속받거나
    # 2. 일단 제외하고 계산하거나
    # 3. 전부 missing인 feature를 빼거나
    
#     this_store_nbr = nan_weather.loc[0,'store_nbr']
    this_station_nbr = nan_weather.loc[0,'station_nbr']
    
#     # 1.
#     if not find_store(this_station_nbr):
#         return nan_weather

#     # 2.
# #     this_station = 
     
    
    print('\t',weather,' EJECTED FROM COLUMN')
    return nan_weather.drop(weather,axis=1)

In [76]:
def partial_missing(nan_weather, weather):
    nan_feature = nan_weather[weather]

    nan_feature[nan_feature.isnull()] = nan_feature.mean()

    nan_weather.loc[:,weather] = nan_feature
    
    print('\t',weather,' : ',nan_feature.mean())
    
    if len(nan_feature[nan_feature.isnull()])==0:
        return nan_weather
    
    print('Error occured when partial_missing() by mean')
    print('station_nbr: ',nan_weather['station_nbr'][0],', feature = ',weather)
    return nan_weather

In [77]:
def replace_by_closest(nan_weather, weather):
    total = len(nan_weather)
    not_missing = len(nan_weather[weather].dropna())
    
    # 1. 아예 없다
    # 다른 station 중 가장 비슷한 feautre를 가진 애
    if not_missing == 0:
        return all_missing(nan_weather, weather)
        
    # 2. 있긴 있다
    # station내에서 가장 비슷한 feature
#     elif 0 < not_missing < total:
#         return partial_missing(nan_weather, weather)
    
    # missing이 없는 경우는 아무것도 안함
    return nan_weather

In [82]:
def drop_all_missing(station_nbr):
    nan_weather = missing_to_nan_weather(station_nbr)
    how_many = how_many_missing(nan_weather)
    
    weathers = set(list(nan_weather.columns)[2:20])
    targets = weathers - {'sunset','sunrise','codesum'}
    
    for weather in targets:
        nan_weather = replace_by_closest(nan_weather,weather)
    
    for each in ['sunset','sunrise','codesum']:
        nan_weather.drop(each, axis=1, inplace=True)
        
    return nan_weather, how_many

In [79]:
tmp,how = drop_all_missing(1)
tmp.drop()

missing_to_nan started !
missing_to_nan completed !
	 depart  EJECTED FROM COLUMN
	 snowfall  EJECTED FROM COLUMN


In [146]:
def filling_missing_by_closest(station_nbr):
    # 빼기 전 all_missing인 feature,
    # sunset,sunrise,codesum과 같은 비연속 데이터는 제외

    weather, how = drop_all_missing(station_nbr)

    no_nan = weather.dropna(axis=0, how='any')
    nan_index = list(set(weather.index) - set(no_nan.index))
    nan = weather.loc[nan_index, :]

    feature = list(weather.columns)[2:-3]

    table = weather.describe()

#     print(feature)

    for each in nan.index:
        nan_row = nan.loc[each, :]
        nan_z = pd.Series([(nan_row[f] - table[f]['mean']) /
                           table[f]['std'] for f in feature])
        # 얘가 nan이면 그 feature는 missing value임

        distance = dict()

        for idx in no_nan.index:
            intact = no_nan.loc[idx, :]
            intact_z = pd.Series(
                [(intact[f] - table[f]['mean']) / table[f]['std'] for f in feature])

            diff = abs(nan_z - intact_z)
            diff = diff.mean()

            distance[idx] = diff
            
        best_close_idx = min(distance, key=distance.get)
        missing_feature = nan_row.isnull()

        nan_row[missing_feature] = no_nan.loc[best_close_idx, missing_feature]
        weather.loc[each, :] = nan_row

    return weather

In [140]:
tmp = pd.Series([1,2,3,np.nan])

In [141]:
tmp.mean()

2.0

In [148]:
best_close = filling_missing_by_closest(1)

missing_to_nan started !
missing_to_nan completed !
	 depart  EJECTED FROM COLUMN
	 snowfall  EJECTED FROM COLUMN
5
1029
9
528
17
532
21
23
535
538
539
29
30
545
34
548
41
45
47
48
560
51
55
574
72
73
74
587
593
83
85
601
90
603
606
97
98
99
609
101
611
615
619
622
114
115
120
633
127
648
650
653
654
145
658
660
149
664
670
671
160
161
672
165
168
173
686
178
692
693
186
698
188
191
704
196
709
198
199
200
710
202
711
712
207
719
720
211
214
216
730
734
226
739
230
742
232
745
750
751
752
755
756
249
762
767
768
772
777
269
785
788
790
281
282
793
794
798
799
800
806
302
303
305
328
335
848
337
849
852
853
854
859
861
350
862
863
353
354
865
356
868
873
363
364
365
876
367
368
881
882
371
374
887
888
377
890
892
893
898
387
390
391
906
399
911
401
912
913
404
405
408
409
410
412
924
925
416
928
929
931
421
933
935
936
425
427
428
942
944
945
946
947
949
439
445
446
957
451
453
460
469
471
983
984
481
995
997
487
1003
1004
1010
500
1012
503
1020
1021


In [150]:
len(best_close)

1035

In [92]:
abs(pd.Series([-10,2]) - pd.Series([-6,1]))

0    4
1    1
dtype: int64

In [94]:
len(pd.Series([1,1,1,np.nan]).dropna())

3

In [95]:
aa = pd.Series([1,1,1,np.nan])

In [98]:
aa[aa.isnull()] = 3

In [99]:
aa

0    1.0
1    1.0
2    1.0
3    3.0
dtype: float64

In [52]:
table = dropped.describe()

In [54]:
table

,station_nbr,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
count,818.0,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000,818.000000
mean,1.0,62.068460,42.852078,52.706601,39.682152,46.568460,14.569682,2.276284,0.119132,29.857408,30.022127,5.275183,20.278729,6.502445,2012.916870,6.304401,15.816626
std,0.0,18.624294,16.648174,17.308688,18.484019,15.988393,14.606823,4.452051,0.293613,0.221914,0.223712,3.409657,9.732811,3.277073,0.799592,3.288868,8.775348
min,1.0,14.000000,0.000000,8.000000,0.000000,4.000000,0.000000,0.000000,0.000000,28.950000,29.200000,0.200000,1.000000,0.600000,2012.000000,1.000000,1.000000
25%,1.0,47.000000,30.250000,39.000000,25.000000,34.000000,0.000000,0.000000,0.000000,29.700000,29.880000,2.700000,12.000000,4.100000,2012.000000,4.000000,8.000000
50%,1.0,64.000000,43.000000,54.000000,42.000000,48.000000,11.000000,0.000000,0.000000,29.855000,30.030000,4.600000,22.000000,5.900000,2013.000000,6.000000,16.000000
75%,1.0,78.000000,57.000000,67.000000,55.000000,60.000000,26.000000,2.000000,0.070000,30.010000,30.170000,7.200000,28.000000,8.100000,2014.000000,9.000000,23.000000
max,1.0,97.000000,77.000000,86.000000,72.000000,76.000000,57.000000,21.000000,3.450000,30.590000,30.770000,19.300000,36.000000,19.800000,2014.000000,12.000000,31.000000


In [57]:
table['tmax']['mean']

62.068459657701709

In [50]:
feeling_missing_by_closest(1)

missing_to_nan started !
missing_to_nan completed !
	 depart  EJECTED FROM COLUMN
	 snowfall  EJECTED FROM COLUMN
['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']


217

In [38]:
dropped.index

Int64Index([   0,    1,    2,    3,    4,    6,    7,    8,   10,   11,
            ...
            1024, 1025, 1026, 1027, 1028, 1030, 1031, 1032, 1033, 1034],
           dtype='int64', length=818)

In [22]:
dropped=tmp.dropna(axis=0,how='any')

In [27]:
index = list(set(tmp.index) - set(dropped.index))

In [28]:
tmp.loc[index,:]

,station_nbr,date,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,sunrise,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
5,1,2012-01-06,46.0,25.0,36.0,21.0,29.0,29.0,0.0,-,...,SN,NaN,29.68,29.84,0.3,1.0,2.4,2012,1,6
1029,1,2014-10-26,58.0,42.0,50.0,40.0,46.0,15.0,0.0,-,...,RA,NaN,29.36,29.54,8.6,27.0,10.3,2014,10,26
9,1,2012-01-10,47.0,29.0,38.0,24.0,32.0,27.0,0.0,-,...,,NaN,29.69,29.86,5.4,25.0,6.5,2012,1,10
528,1,2013-06-12,70.0,57.0,64.0,53.0,57.0,1.0,0.0,-,...,RA,NaN,29.49,29.68,6.5,30.0,7.7,2013,6,12
17,1,2012-01-18,50.0,19.0,35.0,15.0,29.0,30.0,0.0,-,...,RA,NaN,29.55,29.79,13.7,29.0,15.9,2012,1,18
532,1,2013-06-16,79.0,56.0,68.0,52.0,59.0,0.0,3.0,-,...,RA,NaN,29.70,29.84,6.2,22.0,6.7,2013,6,16
21,1,2012-01-22,28.0,7.0,18.0,12.0,18.0,47.0,0.0,-,...,,NaN,30.34,30.54,3.4,4.0,3.7,2012,1,22
23,1,2012-01-24,55.0,41.0,48.0,39.0,44.0,17.0,0.0,-,...,BR,NaN,29.73,29.92,6.8,25.0,8.3,2012,1,24
535,1,2013-06-19,76.0,55.0,66.0,50.0,57.0,0.0,1.0,-,...,BR,NaN,29.85,30.03,1.9,36.0,4.2,2013,6,19
538,1,2013-06-22,86.0,64.0,75.0,59.0,65.0,0.0,10.0,-,...,RA,NaN,30.02,30.18,7.4,22.0,7.7,2013,6,22
